# Phase II Model Test
First attempt at a Phase II model, one which creases the master schedule and assigns students to courses simultaneously.

$S$ -- Set of all students

$C$ -- Set of all courses

$T$ -- Set of all periods {1,2,3,4,7,8} *smaller in this test model*

$I$ -- Set of all instructors

**Variables:**

$x_{i,j}$ for  $i \in S, j \in C$ -- Binary, 1 if student $i$ assigned to course $j$ 

$c_{j,t}$ for $j \in C, t \in T$ -- Binary, 1 if course $j$ to be offered in period $t$

** Parameters:**

$P_{i,j}$ -- Preference for student $i$ on course $j$

$S_i$ -- Seniority constant, e.g., higher for seniors

$D_{i,j}$ -- Binary, 1 if course $i$ and $j$ are in the same department, i.e., if they meet the same requirement, e.g., highschool math

$Ta_{i,j}$ -- Binary, 1 if teacher $i$ is teaching course $j$

$Cap_j$ -- Capacity of course $j$

$Min_j$ -- Minimum number of students needed for course $j$


** Constraints: **

$\sum_{j} x_{i,j} =2 \quad \forall i \in S$ -- Says students can be assigned to two courses (full course load).

$\sum_{j} x_{i,j}c_{j,t} \quad \forall i \in S, t \in T$ -- Students assigned to at most one course per period (now this in conjunction with the previous constraint ensures it is exactly one per period).

$\sum_{i \in C} \sum_{j \neq i \in C} D_{i,j} x_{k,i} = 0 \quad \forall k \in S$ -- Says that students are not assigned to more than 1 class in each department (could always change to more nuanced number of courses per department)

$\sum_{t \in T} c_{j,t} = 1 \quad \forall j \in C$ -- Says that each course taught only once

$\sum_{i \in S} x_{i,j} \leq Cap_j \quad \forall j \in C$ -- Course capacity constraint

$\sum_{j \in C} c_{j,t} Ta_{k,j} \leq 1  \quad \forall k \in I, \forall t \in T$ -- Teacher constraint (a teacher can teach at most one course per period), where $Ta_{k,j}$ is a parameter, not a variable

** Objective: **

$ \text{max }\sum_{i \in S} \sum_{j \in C} x_{i,j} P_{i,j} $ -- Assuming preferences take a higher value if they are a student's preferred choice, this will give a higher weight to higher assignments (at this point, I am leaving out the seniority multiplier).

In [97]:
from pyscipopt import Model, quicksum
import numpy as np

In [98]:
# set up some fake data
## Sets
S = [0,1,2,3,4,5] # 6 students
C = [0,1,2,3] # 4 courses
T = [0,1] # 2 periods
I = [0,1] # 2 instructors

## Preferences
#     0    1    2    3  -- Courses
P = [[1,   0,   2,   1], # student 0
     [0,   2,   1,   1], # student 1
     [1,   2,   0,   1], # student 2
     [1,   1,   0,   2], # student 3
     [1,   1,   1,   0], # student 4
     [0,   2,   1,   1]] # student 5

#     0    1    2    3  -- Courses
# P = [[1,   1,   1,   1], # student 0
#      [1,   1,   1,   1], # student 1
#      [1,   1,   1,   1], # student 2
#      [1,   1,   1,   1], # student 3
#      [1,   1,   1,   1], # student 4
#      [1,   1,   1,   1]] # student 5

## Capacity
#Cap = [5, 5, 5, 5]
Cap = [4]*4
Min = [2]*4

## Course Proximity
# Lets say courses 0 and 1 are in the same department, and a student must have 
# at most one of these courses (or exactly one if the constraint is == 1)
#     0    1    2    3  -- Courses
D = [[0,   0,   1,   0], # 0 
     [0,   0,   0,   1], # 1
     [1,   0,   0,   0], # 2
     [0,   1,   0,   0]] # 3

D = [[0,   1,   0,   0], # 0 
     [1,   0,   0,   0], # 1
     [0,   0,   0,   0], # 2
     [0,   0,   0,   0]] # 3

# Check that it is symetric
if not np.array_equal(np.transpose(D), D):
    raise ValueError("D matrix is not symetric")

## Teacher Assignments
#     0    1    2    3  -- Courses
Ta = [[1,   0,   1,  0],  # Teacher 1
      [0,   1,   0,  1]]  # Teacher 2

In [99]:
# Setup model
m = Model("test")

In [100]:
# Add Student Variables
X = {} # Variable dictionary
for i in S:
    for j in C:
            name = "Student " + str(i) + ", in course " + str(j)
            X[i,j] = m.addVar(name, vtype = 'B')

In [101]:
# Add Course Variable
Course = {} # Variable dictionary
for j in C:
    for t in T:
        name = "Course " + str(j) + " in period " + str(t)
        Course[j,t] = m.addVar(name, vtype='B')

In [102]:
# Add Student assignment constraint (must have two classes)
for i in S:
        m.addCons(quicksum(X[i,j] for j in C) == 2)

In [103]:
# Add student period constraint
for i in S:
    for t in T:
        m.addCons(quicksum(X[i,j]*Course[j,t] for j in C) <= 1)

In [104]:
# Add capacity and minimum constraint
for j in C:
    m.addCons(quicksum(X[i,j] for i in S) <= Cap[j])
    m.addCons(quicksum(X[i,j] for i in S) >= Min[j])

In [105]:
# Add course proximity constraint (without quicksum)
if not np.array_equal(D, np.zeros(np.array(D).shape)):
    for k in S:
        expr = 0 # reset expression (should not have impact)
        for i in C:
            small_set = list(set(C) - set([int(i)])) # C - {i} list of courses without course i
            for j in small_set:
                expr = expr + D[i][j]*X[k,i]  
        m.addCons(expr <= 1)

In [106]:
# Teacher Constraint
for k in I:
    for t in T:
        m.addCons(quicksum(Course[j,t]*Ta[k][j] for j in C) == 1)

In [107]:
# Course Taught only once Constraint
for j in C:
    m.addCons(quicksum(Course[j,t] for t in T) == 1)

In [108]:
# Set objective
m.setObjective(quicksum(X[i,j]*P[i][j] for i in S for j in C), "maximize")

In [109]:
# Solve model
m.optimize()

In [110]:
# Look at output
if m.getStatus() == "optimal":
    # Display which courses each student is assigned to
    for i in S:
        for j in C:
            s = str(X[i,j])
            if m.getVal(X[i,j]) == 1:
                print(s + ":\tyes")
            else:
                print(s + ":\tno")
                #pass
        print("\n")

    # Display which periods courses are assigned to
    for j in C:
        for t in T:
            s = str(Course[j,t])
            if m.getVal(Course[j,t]) == 1:
                print("Course " + str(j) + " to be taught in period", str(t))
                
    # Display the enrollment totals for each course
    print("\nCapacities:")
    for j in C:
        size = 0
        for i in S:
            if m.getVal(X[i,j]) == 1:
                size += 1
        #print("Course", j, "has", size, "seats filled of a possible", Cap[j])
        print("Course", str(j)+":", str(size) + "/" + str(Cap[j]))

else:
    print("The model is", m.getStatus())

Student 0, in course 0:	yes
Student 0, in course 1:	no
Student 0, in course 2:	yes
Student 0, in course 3:	no


Student 1, in course 0:	no
Student 1, in course 1:	yes
Student 1, in course 2:	yes
Student 1, in course 3:	no


Student 2, in course 0:	no
Student 2, in course 1:	yes
Student 2, in course 2:	no
Student 2, in course 3:	yes


Student 3, in course 0:	yes
Student 3, in course 1:	no
Student 3, in course 2:	no
Student 3, in course 3:	yes


Student 4, in course 0:	yes
Student 4, in course 1:	no
Student 4, in course 2:	yes
Student 4, in course 3:	no


Student 5, in course 0:	no
Student 5, in course 1:	yes
Student 5, in course 2:	yes
Student 5, in course 3:	no


Course 0 to be taught in period 1
Course 1 to be taught in period 1
Course 2 to be taught in period 0
Course 3 to be taught in period 0

Capacities:
Course 0: 3/4
Course 1: 3/4
Course 2: 4/4
Course 3: 2/4


## Current issues:
- if I make the number of courses required for each student == 2 the courses are all taugh in the same period
- if I make the number of courses required for each student <= 2 it looks okay
- ** Therefore, the teacher constraint must be wrong **
    - made a small patch, in this issue made the teacher constraint == 1 (so they have to teach a course, it was <= 0 before), and required each student to take 2 courses, seems okay?
    - It was *okay*, just wasn't checking to make sure solution I was looking at was optimal
- Course Caps are being exceeded
    - I think the caps aren't the problem, but the proximity constraint (if I keep the small caps, but do not include the proximity constraint, then the problem has a solution)
    - ** The proximity constraints is THE problem **
    - I added an if statment to including that constraint, only if they are not all zeros
    - This is *okay* it wasn't the constraint as much as it was the data I was feeding in that was too restricitive, making the problem infeasible
- It would be interesting to think about an "eveness" constraint, so classes have moderated sizes"
- If I make the proximity constraint <=1 then students are getting assigned to 2 courses in the same period
    - need new constraint to prohibit this
    - **Done**
- Regarding proximity, I think it might make sense, be easiest to have a proximity matrix *for each subject*, and, as a result, have a constraint for each subject as well